## Step 9 - Run the application on the Production Centre

Step 9 of the tutorial uses a Jupyter Notebook to query the catalog for a Sentinel-1 GRD product, creates a Web Processing Service (WPS) request invoking the data transformation application that was deployed in **Step 8**, monitors the WPS request execution and finally retrieves the data transformation execution results

* First do the imports of the Python libraries required

In [1]:
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import sys
import os
sys.path.append('/opt/anaconda/bin/')
import lxml.etree as etree
import requests
import cioppy
ciop = cioppy.Cioppy()
import shapely
import dateutil.parser
from shapely.wkt import loads 
import pandas as pd
import geopandas as gp
from datetime import datetime, timedelta
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import gdal
import numpy as np

%matplotlib inline

* Define the search parameters: the catalog series OpenSearch endpoint, the time of interest and the area of interest

In [2]:
series = 'https://catalog.terradue.com/sentinel1/search'

start_date = '2017-09-01T00:00:00'
stop_date = '2017-12-10T23:59:59'

geom = 'MULTIPOLYGON (((26.832 9.5136, 28.6843 9.5136, 28.6843 7.8009, 26.832 7.8009, 26.832 9.5136)), ((32.0572 12.4549, 33.9087 12.4549, 33.9087 10.7344, 32.0572 10.7344, 32.0572 12.4549)), ((-5.5 17.26, -1.08 17.26, -1.08 13.5, -5.5 13.5, -5.5 17.26)), ((12.9415 13.7579, 14.6731 13.7579, 14.6731 12.0093, 12.9415 12.0093, 12.9415 13.7579)))'

* Search for Sentinel-1 GRD products in one of the polygons of the area of interest

In [3]:
wkt = loads(geom)[0].wkt

In [4]:
search_params = dict([('geom', wkt),
                     ('start', start_date),
                     ('stop', stop_date),
                      ('pt', 'GRD')])

In [5]:
search = ciop.search(end_point = series,
                     params = search_params,
                     output_fields='self,enclosure,identifier', 
                     model='GeoTime')

* List the Sentinel-1 GRD products found

In [7]:
for index, elem in enumerate(search):
    print(index, elem['identifier'])

(0, 'S1A_IW_GRDH_1SDV_20171207T035045_20171207T035110_019591_021463_A5A3')
(1, 'S1A_IW_GRDH_1SDV_20171207T035016_20171207T035045_019591_021463_0026')
(2, 'S1B_IW_GRDH_1SDV_20171206T035811_20171206T035836_008593_00F41C_389F')
(3, 'S1B_IW_GRDH_1SDV_20171206T035746_20171206T035811_008593_00F41C_DC3A')
(4, 'S1B_IW_GRDH_1SDV_20171201T034954_20171201T035019_008520_00F1CA_204A')
(5, 'S1B_IW_GRDH_1SDV_20171201T034929_20171201T034954_008520_00F1CA_DB5D')
(6, 'S1A_IW_GRDH_1SDV_20171125T035046_20171125T035111_019416_020EED_F4FC')
(7, 'S1A_IW_GRDH_1SDV_20171125T035017_20171125T035046_019416_020EED_2508')
(8, 'S1B_IW_GRDH_1SDV_20171124T035811_20171124T035836_008418_00EE8B_68B5')
(9, 'S1B_IW_GRDH_1SDV_20171124T035746_20171124T035811_008418_00EE8B_5E11')
(10, 'S1A_IW_GRDH_1SDV_20171113T035046_20171113T035111_019241_020964_DCB1')
(11, 'S1A_IW_GRDH_1SDV_20171113T035017_20171113T035046_019241_020964_7E05')
(12, 'S1B_IW_GRDH_1SDV_20171112T035811_20171112T035836_008243_00E941_9FD2')
(13, 'S1B_IW_GRDH_1SDV

* Select the Sentinel-1 GRD product to process

In [7]:
s1_index = 2

* Connect to the WPS server

In [36]:
wps_url = 'http://maxent.wur.nextgeoss.terradue.com/wps/WebProcessingService'
wps_url = 'http://gridded-data.armines.nextgeoss.terradue.com/wps/WebProcessingService'

wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

* Do a GetCapabilities WPS request and list the process:

In [37]:
wps.getcapabilities()

In [38]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm')


* Select the **com.terradue.wps_oozie.process.OozieAbstractAlgorithm** process and print the title and abstract after having submited a WPS DescribeProcess request

In [39]:
process_id = wps.processes[0].identifier

In [40]:
process = wps.describeprocess(process_id)

In [41]:
process.title

'McCloud Map test WPS'

In [42]:
process.abstract

'TODO'

* List the WPS process inputs:

In [43]:
for input in process.dataInputs:
    print(input.identifier)

bbox
width
height
datex
dt
count
min_tile_width
min_tile_height


* Create a Python dictionary with the inputs:

In [44]:
bbox = '5.0,46.0,6.0,45.0'
width = '1009'
height = '1013'
datex = '2453385.900000'
dt = '0.00069444'
count = '3'
min_tile_width = '-1'
min_tile_height = '-1'

In [45]:
inputs = [('bbox', bbox),
        ('width', width),
        ('height', height),
        ('datex', datex),
        ('dt', dt),
        ('count', count),
        ('min_tile_width', min_tile_width),
        ('min_tile_height', min_tile_height)]

* Submit the Execute WPS request:

In [46]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('result_osd', False)])

execution_response = execution.submitRequest(etree.tostring(execution_request))

execution.parseResponse(execution_response)

* Monitor the request:

In [48]:
execution.statusLocation

'http://gridded-data.armines.nextgeoss.terradue.com/wps/RetrieveResultServlet?id=048af2a9-8cae-415b-bf53-ab05f5c8658f'

In [54]:
monitorExecution(execution)

* Check the outcome of the processing request

In [55]:
execution.isSucceded()

True

* Search for the results produced

In [56]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [82]:
for output in execution.processOutputs:
    print(output.data)

['<ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" href="http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180830000011126-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>']


In [97]:
execution.processOutputs[0].data[0]

'<ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" href="http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180830000011126-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>'

In [87]:
for output in execution.processOutputs:
    owslib.wps.printInputOutput(output)

 identifier=result_osd, title=OpenSearch Description to the Results, abstract=None, data type=ComplexData
 Default Value: None 
 reference=None, mimeType=application/xml
 Data Value: <ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" href="http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180830000011126-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>


In [ ]:
results_osd = execution.processOutputs[0].data[0]

In [100]:
print(results_osd)

<ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" href="http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180830000011126-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>


Get the href from the previsous Reference value

In [102]:
referenceUrl = 'http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180830000011126-oozie-oozi-W/results/description'

In [103]:
search_results = ciop.search(end_point=referenceUrl,
                         params=[],
                         output_fields='identifier,enclosure',
                         model='GeoTime')

In [104]:
for index, elem in enumerate(search_results):
    print(elem['enclosure'])

http://gridded-data.armines.nextgeoss.terradue.com/data/run/mccloud_map_wps/0000000-180830000011126-oozie-oozi-W/_results/result.h5
